In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Updated Sentiment Analysis
This notebook follows this tutorial: https://github.com/bentrevett/pytorch-sentiment-analysis.

# Load Pre-Trained Embeddings Manually

In [2]:
from torchtext.vocab import Vectors, Vocab

text = "I love this film very much ."
with open('vector_cache/test-vecs.txt', 'w', encoding='utf-8') as f:
    for token in text.split():
        vec = ["%.6f" % vi for vi in np.random.randn(5)]
        f.write(" ".join([token] + vec))
        f.write("\n")

vecs = Vectors('test-vecs.txt', cache='vector_cache')
print(vecs.stoi)
print(vecs.itos)
print(vecs.vectors)

{'I': 0, 'love': 1, 'this': 2, 'film': 3, 'very': 4, 'much': 5, '.': 6}
['I', 'love', 'this', 'film', 'very', 'much', '.']
tensor([[ 3.1569, -0.4394,  1.0063,  0.6507, -0.2077],
        [ 1.4274,  1.6603, -0.7758, -1.1524,  0.4467],
        [-1.1562,  0.1183, -1.0718, -0.2059, -0.8509],
        [-1.0822, -0.2817, -0.7277,  0.2744,  0.1813],
        [ 0.1380,  0.9525,  0.5184, -0.3377,  0.8099],
        [-0.1883,  0.3074, -0.0763,  1.8269, -0.6083],
        [ 0.5368,  0.3976,  0.0721,  1.0065, -0.4069]])


In [3]:
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator

# Set `include_lengths=True` to return the text lengths too. 
TEXT = Field(tokenize='spacy', include_lengths=True)
GENDER = Field()
LABEL = LabelField(dtype=torch.float)

# Create a Dataset
fields = [('text', TEXT), ('gender', GENDER), ('label', LABEL)]
raw_data = [["I like this film.", "f", "pos"], 
            ["I hate it.", "f", "neg"], 
            ["I have no feelings about it.", "m", "neg"], 
            ["It is my best.", "m", "pos"], 
            ["My father loves it so much and I do think so.", "f", "pos"]]

examples = [Example.fromlist(d, fields) for d in raw_data]
data = Dataset(examples, fields)
data

In [4]:
TEXT.build_vocab(data)
GENDER.build_vocab(data)
LABEL.build_vocab(data)

print(TEXT.vocab.vectors)

None


In [5]:
# The missing tokens are initialized as zeros, or by `unk_init` if provided. 
TEXT.build_vocab(data, vectors=vecs)
print(TEXT.vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.5368,  0.3976,  0.0721,  1.0065, -0.4069],
        [ 3.1569, -0.4394,  1.0063,  0.6507, -0.2077],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-1.0822, -0.2817, -0.7277,  0.2744,  0.1813],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 

In [6]:
BATCH_SIZE = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
iterator = BucketIterator(data, batch_size=BATCH_SIZE, device=device, shuffle=True)

# `batch.text` is now a tuple with the second element being the text lengths.
for batch in iterator:
    print(batch.text)

(tensor([[ 3,  7],
        [16, 12],
        [22, 19],
        [13,  4],
        [ 8,  5],
        [ 4, 20],
        [ 2,  9],
        [ 1,  3],
        [ 1, 11],
        [ 1, 23],
        [ 1,  5],
        [ 1,  2]], device='cuda:0'), tensor([ 7, 12], device='cuda:0'))
(tensor([[ 3,  6],
        [15, 17],
        [ 4, 21],
        [ 2, 10],
        [ 1,  2]], device='cuda:0'), tensor([4, 5], device='cuda:0'))
(tensor([[ 3],
        [18],
        [24],
        [14],
        [ 2]], device='cuda:0'), tensor([5], device='cuda:0'))


# Use Pre-Trained Embeddings

In [7]:
import torchtext

# Set `include_lengths=True` to return the text-lengths too. 
TEXT = Field(tokenize='spacy', include_lengths=True)
LABEL = LabelField(dtype=torch.float)

train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL, root='data')
train_data, valid_data = train_data.split()

In [8]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, 
                 vectors="glove.6B.100d", vectors_cache="vector_cache", 
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [9]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# NOTE: `sort_within_batch=True` for packed padded sequences
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, sort_within_batch=True, device=device)

# Build Model
## Different RNN Architecture: LSTM
## Bidirectional RNN
## Multi-Layer RNN
## Regularization: Dropout

## Implementation Details
### How to ignore the `<pad>` token?
* Pass the index of `<pad>` token as `padding_idx` argument to `nn.Embedding`
* Pass packed padded sequences to RNNs
* Attention?
* `ignore_index` of loss function?

### The `hidden` output of LSTM
The `hidden` has a shape of **(num_layers * num_directions, batch_size, hid_dim)**.  
These are ordered in **(forward_layer_0, backward_layer_0, forward_layer_1, backward_layer_1, ..., forward_layer_n, backward_layer_n)**, from bottom to top.  

In [10]:
class Classifier(nn.Module):
    def __init__(self, in_dim, emb_dim, hid_dim, out_dim, n_layers, bidirect, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, 
                           bidirectional=bidirect, dropout=dropout)
        # `bidirectional=True` by default
        self.fc = nn.Linear(hid_dim*2, out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lens):
        # text: (step, batch)
        embedded = self.dropout(self.emb(text))
        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lens)
        # hidden: (num_layers*num_directions, batch, hid_dim)
        packed_outs, (hidden, cell) = self.rnn(packed_embedded)
        # Unpack sequence, NOT used here. 
        outs, out_lens = nn.utils.rnn.pad_packed_sequence(packed_outs)

        hidden = self.dropout(torch.cat([hidden[-2], hidden[-1]], dim=-1))
        return self.fc(hidden)

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
HID_DIM = 256
OUT_DIM = 1
N_LAYERS = 2
BIDIRECT = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

classifier = Classifier(IN_DIM, EMB_DIM, HID_DIM, OUT_DIM, 
                       N_LAYERS, BIDIRECT, DROPOUT, PAD_IDX).to(device)
count_parameters(classifier)

4810857

### Initialize Embeddings with Pre-Trained Vectors
* Replace the embedding layer's weight with `TEXT.vocab.vectors`.  
    * **Note:** Use `weight.data` for data Initialization.  
* Explicitly Initialize the embeddings of `<unk>` and `<pad>` tokens to zeros.  
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [12]:
print(TEXT.vocab.vectors.size())
print(classifier.emb.weight.size())

print(TEXT.vocab.vectors[:5, :8])
print(classifier.emb.weight[:5, :8])
classifier.emb.weight.data.copy_(TEXT.vocab.vectors)
print(classifier.emb.weight[:5, :8])

torch.Size([25002, 100])
torch.Size([25002, 100])
tensor([[-0.3902,  1.4256,  0.0491, -0.0559, -1.0172,  0.6198,  0.3173, -1.8878],
        [-0.4843,  1.6366, -1.4119, -0.4014, -1.1278,  1.6279,  0.7653, -0.5195],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]])
tensor([[-0.6656,  1.2448, -0.8936, -1.0469, -2.2802, -0.2518, -1.0071, -1.1899],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0271, -0.6303, -0.5369, -1.6774, -1.1523, -1.9146,  0.6415, -0.3668],
        [ 0.2094, -0.7820, -0.1052, -1.0747, -2.5509, -1.2299, -0.0707,  0.0882],
        [-0.7804,  0.8003,  1.5012,  2.2774, -0.6308,  2.0983,  0.8358,  0.0679]],
       device='cuda:0', grad_fn=<SliceBackward>)
tensor([[-0.3902,  1.4256,  0.0491, -0.0559, -1.0172,  0.6198,  0.3173, -1.8878

In [13]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

classifier.emb.weight.data[UNK_IDX].zero_()
classifier.emb.weight.data[PAD_IDX].zero_()

print(classifier.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]],
       device='cuda:0', grad_fn=<SliceBackward>)


# Train Model

In [14]:
optimizer = optim.Adam(classifier.parameters())
# Binary cross entropy with logits. 
# The binary version of cross entropy loss. 
loss_func = nn.BCEWithLogitsLoss().to(device)

In [15]:
def train_epoch(classifier, iterator, optimizer, loss_func):
    classifier.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifier(text, text_lens).squeeze(-1)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifier, iterator, loss_func):
    classifier.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifier(text, text_lens).squeeze(-1)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [16]:
import time
N_EPOCHS = 5
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train_epoch(classifier, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(classifier.state_dict(), 'models/tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 32s
	Train Loss: 0.643 | Train Acc: 61.78%
	 Val. Loss: 0.571 |  Val. Acc: 72.95%
Epoch: 02 | Epoch Time: 0m 32s
	Train Loss: 0.604 | Train Acc: 66.72%
	 Val. Loss: 0.606 |  Val. Acc: 68.69%
Epoch: 03 | Epoch Time: 0m 32s
	Train Loss: 0.578 | Train Acc: 70.24%
	 Val. Loss: 0.400 |  Val. Acc: 82.88%
Epoch: 04 | Epoch Time: 0m 32s
	Train Loss: 0.388 | Train Acc: 83.36%
	 Val. Loss: 0.333 |  Val. Acc: 86.22%
Epoch: 05 | Epoch Time: 0m 32s
	Train Loss: 0.317 | Train Acc: 86.82%
	 Val. Loss: 0.314 |  Val. Acc: 86.79%


In [17]:
classifier.load_state_dict(torch.load('models/tut2-model.pt'))

valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
test_loss, test_acc = eval_epoch(classifier, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Val. Loss: 0.314 | Val. Acc: 86.79%
Test Loss: 0.325 | Test Acc: 86.15%


# User Input

In [18]:
import spacy
nlp = spacy.load('en')

def predict_senti(classifier, sentence):
    classifier.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[tok] for tok in tokenized]
    lens = len(indexed)

    indexed = torch.tensor(indexed, dtype=torch.long).unsqueeze(1).to(device)
    lens = torch.tensor(lens, dtype=torch.long).unsqueeze(0).to(device)
    pred = torch.sigmoid(classifier(indexed, lens)).round().type(torch.long)
    return LABEL.vocab.itos[pred.item()]

In [19]:
predict_senti(classifier, "This is a good film.")

'pos'

In [20]:
predict_senti(classifier, "This film is terrible.")

'neg'

# Check Embeddings
* The Embeddings of `unk` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [21]:
print(classifier.emb.weight[:5, :8])

tensor([[ 0.0363, -0.0296,  0.0073, -0.0088, -0.0543, -0.0388, -0.0570,  0.0349],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0600, -0.3097,  0.7223, -0.4018,  0.1127,  0.0485, -0.3875,  0.3356],
        [-0.0983, -0.0396,  0.5816, -0.5624,  0.6413,  0.0653,  0.0712,  0.3738],
        [-0.4561,  0.1625,  0.4549, -0.5850, -0.3922,  0.0096,  0.1548,  0.1458]],
       device='cuda:0', grad_fn=<SliceBackward>)
